# 01 输入模块

首先，单个或批量文本/提示被输入模型。例如：图中的"Hello World"。输入模型的必须是数字格式，因为模型无法直接处理文本。分词器将这些文本/提示转换为标记ID（词汇表中标记的索引号表示）。

我们将使用Tiny Shakespeare数据集构建词汇表并训练模型。Llama 3模型使用TikToken作为分词器，这是一种子词分词器。但是我们这个实现将使用字符级分词器。这样做的主要原因是让我们能够自行构建词汇表和分词器，包括编码和解码函数，这样可以深入理解底层工作原理并完全掌控代码。每个标记ID将被转换为128维的嵌入向量（原始Llama 3 8B中为4096维）。

然后这些嵌入将被传递到下一个解码器模块。

In [1]:
# 导入必要的库 
import torch 
from torch import nn 
from torch.nn import functional as F 
import math 
import numpy as np 
import time 
from dataclasses import dataclass 
from typing import Optional, Tuple, List 
import pandas as pd 
from matplotlib import pyplot as plt

In [2]:
### 步骤1: 输入模块 ###  
# 使用Tiny Shakespeare数据集实现字符级分词器。部分字符级分词器代码参考自Andrej Karpathy的GitHub仓库
# (https://github.com/karpathy/nanoGPT/blob/master/data/shakespeare_char/prepare.py)
# 加载tiny_shakespeare数据文件 (https://github.com/tamangmilan/llama3/blob/main/tiny_shakespeare.txt)  
device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
# 根据可用性分配设备为cuda或cpu  

In [3]:
# 加载tiny_shakespeare.txt数据文件
with open('tiny_shakespeare.txt', 'r') as f: 
    data = f.read() 

In [5]:
# 通过提取tiny_shakespeare数据中的所有唯一字符准备词汇表 
vocab = sorted(list(set(data))) 

In [6]:
# 训练Llama 3模型需要额外的标记，如<|begin_of_text|>、<|end_of_text|>和<|pad_id|>，将它们添加到词汇表中 
vocab.extend(['<|begin_of_text|>','<|end_of_text|>','<|pad_id|>']) 
vocab_size = len(vocab) 